# **TITLE**

## **Import Library**

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer, StandardScaler, MinMaxScaler, OrdinalEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from scipy import stats
from scipy.stats import zscore
from imblearn.over_sampling import SMOTE

## **Import Data**

In [2]:
df = pd.read_csv('IBM Attrition.csv')

### Data Type Check (David)


### Duplicates Handling (David)

### Missing Values Handling (Reza)

## **Data Understanding / EDA**

### Descriptive Statistics (Reza)

### Univariate Analysis (Reza)

### Multivariate Analysis (Reza)

### Feature Selection (Gibran)

### Feature Engineering (Gibran)

## **Data Preparation / Preprocessing**

### Data Splitting (Erick)

### Outlier Handling (Erick)

### Transformasi (Erick)

#### *Log Transform*

#### *Standardize*

#### *Normalize*

### Encoding (Bila Perlu) (Erick)

### Preprocessing Pipeline (Perlu Discusi dengan Mentor)

### Imbalance Handling (Hanay data Train) (Erick)

## Modeling

### Library For Modeling

### Train (.fit) And Predict (.predict) Baseline Model

### Hyperparameter Tunning

### Final Model

## Model Evaluation

### Test the Prediction of Final Model

### Evaluation Metrics

### Visualitasion Actual vs Prediction (if needed)